<a href="https://colab.research.google.com/github/oymin2001/DataScience/blob/main/%EB%84%A4%EC%9D%B4%EB%B2%84%20%EC%98%81%ED%99%94%EB%A6%AC%EB%B7%B0%20%EA%B0%90%EC%84%B1%EB%B6%84%EC%84%9D%20%EC%A0%84%EC%B2%98%EB%A6%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install konlpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 33.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 32.2 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from konlpy.tag import Okt
import re
import warnings

%matplotlib inline
warnings.filterwarnings('ignore')

In [ ]:
movie = pd.read_csv('movie_rv.csv')
movie.head()

,Unnamed: 0,id,document,label
0,0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,2,10265843,너무재밓었다그래서보는것을추천한다,0
3,3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [ ]:
movie.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  150000 non-null  int64 
 1   id          150000 non-null  int64 
 2   document    149995 non-null  object
 3   label       150000 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 4.6+ MB


In [ ]:
movie.drop('Unnamed: 0',axis=1,inplace=True) # index이므로 제거
movie.dropna(inplace=True) # 5개는 무시할만하므로 제거
len(np.unique(movie.id)) / len(movie.id) # 모두 고유 id이므로 필요없는 데이터

1.0

In [ ]:
movie.drop('id', axis=1,inplace=True)
np.unique(movie.label) # 이진분류 문제

array([0, 1])

In [ ]:
movie.label.value_counts() # 0,1 비율이 거의 균등함

0    75170
1    74825
Name: label, dtype: int64

In [ ]:
movie.document.head(10) # 한글, 공백만 남김

0                                  아 더빙.. 진짜 짜증나네요 목소리
1                    흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나
2                                    너무재밓었다그래서보는것을추천한다
3                        교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정
4    사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...
5        막 걸음마 뗀 3세부터 초등학교 1학년생인 8살용영화.ㅋㅋㅋ...별반개도 아까움.
6                                원작의 긴장감을 제대로 살려내지못했다.
7    별 반개도 아깝다 욕나온다 이응경 길용우 연기생활이몇년인지..정말 발로해도 그것보단...
8                               액션이 없는데도 재미 있는 몇안되는 영화
9        왜케 평점이 낮은건데? 꽤 볼만한데.. 헐리우드식 화려함에만 너무 길들여져 있나?
Name: document, dtype: object

In [ ]:
movie.document = movie.document.str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
movie.document.head(10)

0                                    아 더빙 진짜 짜증나네요 목소리
1                           흠포스터보고 초딩영화줄오버연기조차 가볍지 않구나
2                                    너무재밓었다그래서보는것을추천한다
3                            교도소 이야기구먼 솔직히 재미는 없다평점 조정
4    사이몬페그의 익살스런 연기가 돋보였던 영화스파이더맨에서 늙어보이기만 했던 커스틴 던...
5                막 걸음마 뗀 세부터 초등학교 학년생인 살용영화ㅋㅋㅋ별반개도 아까움
6                                 원작의 긴장감을 제대로 살려내지못했다
7    별 반개도 아깝다 욕나온다 이응경 길용우 연기생활이몇년인지정말 발로해도 그것보단 낫...
8                               액션이 없는데도 재미 있는 몇안되는 영화
9            왜케 평점이 낮은건데 꽤 볼만한데 헐리우드식 화려함에만 너무 길들여져 있나
Name: document, dtype: object

In [ ]:
movie.document[movie.document.str.strip() == ""] # 한글, 공백을 제외하고는 다 제거하였으므로 na값 확인. 1255개가 있음

404           
412           
470           
584           
593           
          ... 
149630        
149718        
149773        
149862        
149966        
Name: document, Length: 1255, dtype: object

In [ ]:
movie.document[movie.document.str.strip() == ""] = np.nan # 공백 제거
movie.dropna(inplace=True)
movie.head()

,document,label
0,아 더빙 진짜 짜증나네요 목소리,0
1,흠포스터보고 초딩영화줄오버연기조차 가볍지 않구나,1
2,너무재밓었다그래서보는것을추천한다,0
3,교도소 이야기구먼 솔직히 재미는 없다평점 조정,0
4,사이몬페그의 익살스런 연기가 돋보였던 영화스파이더맨에서 늙어보이기만 했던 커스틴 던...,1


In [ ]:
X = []
y = movie.label

Okt = Okt() #형태소 분석
for sentence in movie.document:
    X.append(Okt.morphs(sentence, stem=True))

In [ ]:
X[:3] # 맞춤법만 맞으면 괜찮게 잘린것 같다.

[['아', '더빙', '진짜', '짜증나다', '목소리'],
 ['흠', '포스터', '보고', '초딩', '영화', '줄', '오버', '연기', '조차', '가볍다', '않다'],
 ['너', '무재', '밓었', '다그', '래서', '보다', '추천', '한', '다']]

In [ ]:
# 기본 성능 테스트
vectorizer = CountVectorizer(tokenizer=lambda x: x, lowercase=False) # tokenizing + vectorization, lowercase=False로 해주지 않으면 가끔 에러가 난다고함.
X_vanilla = vectorizer.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_vanilla, y, test_size=0.3, random_state=42)
lr_clf = LogisticRegression(random_state=42)
lr_clf.fit(X_train,y_train)
y_pred = lr_clf.predict(X_test)
print('accuracy:{0:.3f}, roc_auc:{1:.3f}'.format(accuracy_score(y_test, y_pred),roc_auc_score(y_test, y_pred))) # 괜찮게 성능이 나옴.
# stopwords, max_features관련하여 전처리 후 성능을 비교해보겠음

accuracy:0.838, roc_auc:0.838


In [ ]:
def get_score(data,label):
    X_train, X_test, y_train, y_test = train_test_split(data, label, test_size=0.3, random_state=42)
    lr_clf = LogisticRegression(random_state=42)
    lr_clf.fit(X_train,y_train)
    y_pred = lr_clf.predict(X_test)
    print('accuracy:{0:.3f}, roc_auc:{1:.3f}'.format(accuracy_score(y_test, y_pred),roc_auc_score(y_test, y_pred)))

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)
sorted(tokenizer.word_counts.items(), key=lambda x: x[1],reverse=True)[:50] #가장 많이 나온 단어 상위 50
# 감성분석과 상관없는 단어들은 지우겠음

[('이', 55799),
 ('영화', 50643),
 ('보다', 41144),
 ('하다', 40877),
 ('의', 30985),
 ('에', 27123),
 ('가', 26718),
 ('을', 23262),
 ('도', 21097),
 ('들', 18163),
 ('는', 16979),
 ('를', 16172),
 ('은', 16057),
 ('없다', 15628),
 ('이다', 13241),
 ('있다', 12462),
 ('좋다', 12143),
 ('너무', 11154),
 ('다', 10181),
 ('정말', 9752),
 ('한', 9641),
 ('재밌다', 9066),
 ('되다', 8886),
 ('적', 8770),
 ('만', 8705),
 ('같다', 8345),
 ('진짜', 8338),
 ('으로', 7940),
 ('로', 7839),
 ('아니다', 7761),
 ('않다', 7728),
 ('점', 7636),
 ('에서', 6982),
 ('만들다', 6510),
 ('과', 6421),
 ('나오다', 6404),
 ('연기', 6328),
 ('평점', 6315),
 ('것', 6302),
 ('최고', 6040),
 ('내', 5963),
 ('그', 5673),
 ('나', 5637),
 ('안', 5502),
 ('인', 5483),
 ('스토리', 5335),
 ('생각', 5315),
 ('못', 5298),
 ('왜', 5125),
 ('드라마', 5063)]

In [ ]:
# 감정과 관련 없는 단어들은 제외하겠음
not_sentimental_words = ['이','영화','보다','하다','의','에','가','을','도','들','는','를','은','이다','되다','적','만','으로','로','점','에서','만들다','과','나오다','연기', \
'평점','것','내','그','안','인','스토리','생각','드라마']

vectorizer = CountVectorizer(tokenizer=lambda x: x, stop_words=not_sentimental_words,lowercase=False)
X_remove_stopwords = vectorizer.fit_transform(X)
get_score(X_remove_stopwords,y) # 성능변화가 없음

accuracy:0.836, roc_auc:0.836


In [ ]:
len(tokenizer.word_index) # 전체 단어 43770개

43770

In [ ]:
cnt = np.zeros(4)
for key, value in tokenizer.word_counts.items():
    cnt[3] += value

    if value == 3:
        cnt[2] += 1
    elif value == 2:
        cnt[1] += 1
    elif value < 2:
        cnt[0] += 1
    else:
        pass

cnt / len(tokenizer.word_index) # 중복제외 1번 등장: 43%, 2번 등장: 13%, 3번 등장: 7%

array([ 0.42789582,  0.12791867,  0.07205849, 43.68169979])

In [ ]:
# 3번 이하 등장한 단어가 중복포함하여 전체 단어중 비율
cnt[0:3].sum() / cnt[3]

0.01437382188218508

In [ ]:
# 1% 정도의 매우 적은 비율을 차지하므로 3개이하는 버리겠음
max_features = int(len(tokenizer.word_index) - (cnt[0]+cnt[1]+cnt[2]))
max_features

16288

In [ ]:
vectorizer = CountVectorizer(tokenizer=lambda x: x, stop_words=not_sentimental_words,max_features=max_features,lowercase=False)
X_fin = vectorizer.fit_transform(X)
get_score(X_fin,y)

accuracy:0.836, roc_auc:0.836


In [ ]:
#최적의 파라미터 찾기
X_train, X_test, y_train, y_test = train_test_split(X_fin, y, test_size=0.3, random_state=42)

model = LogisticRegression(random_state=42)
parameters = {'solver': ['lbfgs','liblinear','newton-cholesky'],
              'penalty':['l1','l2'],
              'C':[1e-1,1,1e1,1e3],
              'max_iter':[100,300,500,700]}

grid_lr = GridSearchCV(model,param_grid = parameters, scoring = 'accuracy')
grid_lr.fit(X_train,y_train)

NameError: ignored

In [ ]:
grid_lr.best_params_

In [ ]:
cvres = grid_lr.cv_results_ #nan값은 몇몇 solver들은 특정 penalty만 사용할 수 있기 때문
for mean_score, params in zip(cvres['mean_test_score'], cvres['params']):
    print(mean_score, params)

In [ ]:
fin_model = grid_lr.best_estimator_
y_pred = fin_model.predict(X_test)
print('accuracy:{0:.3f}, roc_auc:{1:.3f}'.format(accuracy_score(y_test, y_pred),roc_auc_score(y_test, y_pred)))

accuracy:0.835, roc_auc:0.835
